In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-fake-news-dataset/final_en.csv


# Loading Dataset

In [2]:
data = pd.read_csv("/kaggle/input/english-fake-news-dataset/final_en.csv")

In [3]:
data.head(5)

,title,text,lebel
0,Trump backs off praise of Russia's Putin after...,"HENDERSON, Nev. (Reuters) - U.S. Republican pr...",1
1,Trump's funding request for U.S. border wall h...,WASHINGTON (Reuters) - President Donald Trump’...,1
2,"As Votes For Trump Went Up, Canada’s Immigrat...","Well, sad to say, it s a sure chance the next ...",0
3,"U.S. Navy, shipbuilders ready for Trump's expa...","SIMI VALLEY, Calif. (Reuters) - The U.S. arms ...",1
4,"Trump defends DACA move, urges Congress to ena...",WASHINGTON (Reuters) - President Donald Trump ...,1


# **Importing Libraries**

In [115]:
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout, SimpleRNN, Flatten
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

**Spliting the dataset into training and testing datsets**

In [9]:
train_df , test_df = train_test_split(data, test_size=0.2, random_state=42)

print(len(train_df),len(test_df))

8000 2000


# Preprocessing the Dataset

In [173]:
def preprocess(df):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(df['title']+df['text'])
    vocab = len(tokenizer.word_index) + 1
    title_sequences = tokenizer.texts_to_sequences(df['title'])
    news_sequences = tokenizer.texts_to_sequences(df['text'])
    
    title_sequences = pad_sequences(title_sequences,maxlen=20)
    news_sequences = pad_sequences(news_sequences,maxlen=200)
    main_sequence = np.concatenate([title_sequences,news_sequences],axis=1)
    
    y = df['lebel']
    return main_sequence , y, vocab

In [174]:
(train_x ,train_y, vocab) = preprocess(train_df)
(test_x ,test_y,_) = preprocess(test_df)

print(len(train_x),len(test_x))

8000 2000


In [30]:
print(train_y.shape)

(8000,)


# Compiling the model

In [209]:
model = Sequential([
    Embedding(input_dim= vocab, output_dim= 4, input_length= 220),
    Flatten(),
    #LSTM(128,dropout=0.2,recurrent_dropout= 0.2,kernel_regularizer=l2(0.01)),
    Dense(4,activation='relu',kernel_regularizer=l2(0.01)),
    Dropout(0.2),
    Dense(1,activation='sigmoid')
])
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model.compile(
    optimizer= 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [210]:
history = model.fit(train_x,train_y, epochs= 10,validation_data= (test_x,test_y),batch_size= 64, callbacks=[early_stopping] )

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6244 - loss: 0.7068 - val_accuracy: 0.8130 - val_loss: 0.5265
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9011 - loss: 0.3534 - val_accuracy: 0.8320 - val_loss: 0.4534
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9590 - loss: 0.2331 - val_accuracy: 0.8390 - val_loss: 0.4267
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9743 - loss: 0.1907 - val_accuracy: 0.8280 - val_loss: 0.4381
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9795 - loss: 0.1598 - val_accuracy: 0.8395 - val_loss: 0.4156
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9848 - loss: 0.1419 - val_accuracy: 0.8450 - val_loss: 0.4067
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9870 - loss: 0.1256 - val_accuracy: 0.8075 - val_loss: 0.4369
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9864 - loss: 0.1215 - val_accuracy: 0.

# Predicting the testing dataset news

In [212]:
prediction = model.predict(test_x)
predicted_classes = [1 if pred > 0.5 else 0 for pred in prediction]

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [211]:
model.evaluate(test_x,test_y)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8387 - loss: 0.3943


[0.3894246518611908, 0.843999981880188]

In [213]:
print(f'Actual sentiment: {test_y[:10].values}',
      f'predictedt sentiment: {predicted_classes[:10]}')

Actual sentiment: [0 1 0 0 1 1 0 1 0 1] predictedt sentiment: [0, 1, 0, 0, 1, 1, 0, 1, 0, 1]


# Input function for News to predict

In [261]:
user = int(input("Enter the number of news you want to predict"))
print(test_df['text'].values[user])

Enter the number of news you want to predict 0


Fox News   The Five  host, Jesse Watters let loose on liberal crybaby co-host Juan Williams during their most recent show. Waters lost it over liberals who would dare to call President Trump and his family  unpatriotic . Watters started his rant by saying, I m very sick of the Democrats calling the Trump family  unpatriotic!'  He went on to say that the left are  now the McCarthyites .He was only getting started however. Watters then gave Juan Williams a history lesson that just about blew the doors off the Fox News set.Watters to Williams:  Let s remember what party defeated the Soviet Union, while the Democratic Party was trying to appease and mimic Communism. Party. Democrats have been cozying up to dictators and our enemies for years. Giving the Iranians billions of dollars, giving North Koreans billions of dollars. Shaking Hugo Chavez s hand, shaking the Castro s hands.  The Clinton s were taking money from Chinese nationals in 1996 reelection campaign and then selling satellite t

# Prediction

In [262]:
if predicted_classes[user] == 1:
    print("Original News")
else:
    print("Fake News")

Fake News
